# Intro

In [ ]:
!pip install datasets
!pip install transformers

In [2]:
## IMPORTS
import json

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm.notebook import tqdm

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

import datasets

from transformers import (LongformerModel, LongformerTokenizer, LongformerPreTrainedModel,
                          LongformerConfig, Trainer, TrainingArguments, EarlyStoppingCallback)
from transformers.models.longformer.modeling_longformer import LongformerQuestionAnsweringModelOutput

In [ ]:
# GLOBAL VARIABLES

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

UNKNOWN = "unknown"
DATASET_TRAIN_PATH = "/content/drive/MyDrive/Colab Notebooks/Seminar/dataset/coqa_flat_train_df.pkl"
DATASET_TEST_PATH = "/content/drive/MyDrive/Colab Notebooks/Seminar/dataset/coqa_flat_val_df.pkl"
ANSWERS_PATH = "/content/drive/MyDrive/Colab Notebooks/Seminar/answers/{file_name}"

In [ ]:
## HELPER FUNCTIONS

def _get_question_end_index(input_ids, sep_token_id):
    """
    Computes the index of the first occurrence of `sep_token_id`.
    """

    sep_token_indices = (input_ids == sep_token_id).nonzero()
    batch_size = input_ids.shape[0]

    assert sep_token_indices.shape[1] == 2, "`input_ids` should have two dimensions"
    assert (
        sep_token_indices.shape[0] == 3 * batch_size
    ), f"There should be exactly three separator tokens: {sep_token_id} in every sample for questions answering. You might also consider to set `global_attention_mask` manually in the forward function to avoid this error."
    return sep_token_indices.view(batch_size, 3, 2)[:, 0, 1]


def _compute_global_attention_mask(input_ids, sep_token_id, before_sep_token=True):
    """
    Computes global attention mask by putting attention on all tokens before `sep_token_id` if `before_sep_token is
    True` else after `sep_token_id`.
    """
    question_end_index = _get_question_end_index(input_ids, sep_token_id)
    question_end_index = question_end_index.unsqueeze(dim=1)  # size: batch_size x 1
    # bool attention mask with True in locations of global attention
    attention_mask = torch.arange(input_ids.shape[1], device=input_ids.device)
    if before_sep_token is True:
        attention_mask = (attention_mask.expand_as(input_ids) < question_end_index).to(torch.uint8)
    else:
        # last token is separation token and should not be counted and in the middle are two separation tokens
        attention_mask = (attention_mask.expand_as(input_ids) > (question_end_index + 1)).to(torch.uint8) * (
            attention_mask.expand_as(input_ids) < input_ids.shape[-1]
        ).to(torch.uint8)

    return attention_mask

# Model

In [ ]:
class MyLongformerForQuestionAnswering(LongformerPreTrainedModel):

    _keys_to_ignore_on_load_unexpected = [r"pooler"]

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')

        self.longformer = LongformerModel.from_pretrained('allenai/longformer-base-4096', gradient_checkpointing=True)
        self.qa_outputs = nn.Linear(config.hidden_size, config.num_labels)

        # Initialize weights and apply final processing
        self.post_init()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        global_attention_mask=None,
        head_mask=None,
        token_type_ids=None,
        position_ids=None,
        inputs_embeds=None,
        start_positions=None,
        end_positions=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        if global_attention_mask is None:
            if input_ids is None:
                logger.warning(
                    "It is not possible to automatically generate the `global_attention_mask` because input_ids is None. Please make sure that it is correctly set."
                )
            else:
                # set global attention on question tokens automatically
                global_attention_mask = _compute_global_attention_mask(input_ids, self.config.sep_token_id)
 
        outputs = self.longformer(
            input_ids,
            attention_mask=attention_mask,
            global_attention_mask=global_attention_mask,
            head_mask=head_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        sequence_output = outputs[0]

        logits = self.qa_outputs(sequence_output)
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1).contiguous()
        end_logits = end_logits.squeeze(-1).contiguous()

        total_loss = None
        if start_positions is not None and end_positions is not None:
            # If we are on multi-GPU, split add a dimension
            if len(start_positions.size()) > 1:
                start_positions = start_positions.squeeze(-1)
            if len(end_positions.size()) > 1:
                end_positions = end_positions.squeeze(-1)
            # sometimes the start/end positions are outside our model inputs, we ignore these terms
            ignored_index = start_logits.size(1)
            start_positions = start_positions.clamp(0, ignored_index)
            end_positions = end_positions.clamp(0, ignored_index)

            loss_fct = nn.CrossEntropyLoss(ignore_index=ignored_index)
            start_loss = loss_fct(start_logits, start_positions)
            end_loss = loss_fct(end_logits, end_positions)
            total_loss = (start_loss + end_loss) / 2

        if not return_dict:
            output = (start_logits, end_logits) + outputs[2:]
            return ((total_loss,) + output) if total_loss is not None else output

        return LongformerQuestionAnsweringModelOutput(
            loss=total_loss,
            start_logits=start_logits,
            end_logits=end_logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
            global_attentions=outputs.global_attentions,
        )

    def construct_answer(self, outputs, encoding):
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits
        all_tokens = self.tokenizer.convert_ids_to_tokens(encoding["input_ids"][0].tolist())

        start_index = torch.argmax(start_logits)

        end_sorted = torch.argsort(end_logits, descending=True).squeeze().tolist()
        for i in end_sorted:
            if i+1 > start_index:
                end_index = i+1
                break

        if start_index < end_index:
            answer_tokens = all_tokens[start_index:end_index]
            answer = self.tokenizer.decode(self.tokenizer.convert_tokens_to_ids(answer_tokens))
        else: # TODO: a good condition for unknown
            answer = UNKNOWN
        return answer

In [ ]:
model = MyLongformerForQuestionAnswering.from_pretrained('allenai/longformer-base-4096')
model = model.to(device)

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing MyLongformerForQuestionAnswering: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense

In [ ]:
def test_my_longformer_for_question_answering():

    question, text = "Who was Ali?", "Ali was a nice programmer."

    encoding = model.tokenizer(question, text, return_tensors="pt").to(device)

    outputs = model(**encoding)
    # outputs = model(**encoding, start_positions=start_positions, end_positions=end_positions)

    answer = model.construct_answer(outputs, encoding)
    print(answer)

test_my_longformer_for_question_answering()

?</s></s>Ali was a nice programmer.


# Prepare Data

In [ ]:
df_train = pd.read_pickle(DATASET_TRAIN_PATH)
df_test = pd.read_pickle(DATASET_TEST_PATH)

In [ ]:
df_train.head(1)

,index,name,filename,id,source,story,turn_id,question,input_text,span_text,span_start,span_end,bad_turn
0,0,Vatican_Library.txt,Vatican_Library.txt,3zotghdk5ibi9cex97fepx7jetpso7,wikipedia,"The Vatican Apostolic Library (), more commonl...",1,When was the Vat formally opened?,It was formally established in 1475,Formally established in 1475,151,179,False


# Test model with data

```[{
        "id": "3dr23u6we5exclen4th8uq9rb42tel",
        "turn_id": 3,
        "answer": "no"
    }, ...]```

In [ ]:
predictions = []

for index, item in tqdm(df_test.iterrows()):

    question, text = item["question"], item["story"]

    encoding = model.tokenizer(question, text, return_tensors="pt").to(device)

    outputs = model(**encoding)
    # outputs = model(**encoding, start_positions=start_positions, end_positions=end_positions)

    answer = model.construct_answer(outputs, encoding)

    predictions.append(
        {
            "id": item["id"],
            "turn_id": item["turn_id"],
            "answer": answer
        }
    )

0it [00:00, ?it/s]

In [ ]:
# SAVE THE ANSWERS
file_name = "longformer-without-training-dev-v0.json"
file_path = ANSWERS_PATH.format(file_name=file_name)
json_file = open(file_path, "w")
json_file.write(json.dumps(predictions))
json_file.close()